In [ ]:
import os
import socket
from pyspark.sql import SparkSession

# ดึง IP ของ Container ตัวเอง (Jupyter)
hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)
print(f"🔹 Jupyter IP: {local_ip}")

# --- CONFIGURATION ---
MINIO_ENDPOINT = "http://minio:9000"
HIVE_METASTORE_URI = "thrift://docker-compose-hive-metastore-1:9083" # เช็คชื่อ Container Hive ดีๆ
ACCESS_KEY = "minio"
SECRET_KEY = "minio123"

PACKAGES = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0"
]

spark = (
    SparkSession.builder
    .appName("Jupyter-Iceberg-Cluster")
    
    # ชี้ไปที่ Spark Master (ใช้ชื่อ service ได้เลยเพราะอยู่ใน network เดียวกัน)
    .master("spark://spark-master:7077")
    
    # ---------------------------------------------------------
    # ✅ ส่วนสำคัญที่ต้องเพิ่มเพื่อแก้ Network Issue
    # 1. บอก Worker ว่า Driver อยู่ที่ IP ไหน
    .config("spark.driver.host", local_ip) 
    
    # 2. บังคับใช้ Port ที่เราเปิดไว้ใน Docker Compose
    .config("spark.driver.port", "20020")
    .config("spark.blockManager.port", "20021")
    
    # 3. ให้ Driver ฟังทุก Interface ใน Container ตัวเอง
    .config("spark.driver.bindAddress", "0.0.0.0")
    # ---------------------------------------------------------

    .config("spark.jars.packages", ",".join(PACKAGES))
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

    # S3 Config
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
    # Iceberg Config
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.type", "hive")
    .config("spark.sql.catalog.iceberg.uri", HIVE_METASTORE_URI)
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/iceberg_data")
    
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark Session Created with Fixed Ports!")

# ทดสอบ
spark.sql("SELECT * FROM iceberg.demo_schema.jupyter_users").show()
print("ok")

In [ ]:
import os
import socket
from pyspark.sql import SparkSession

# ดึง IP ของ Container ตัวเอง (Jupyter)
hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)
print(f"🔹 Jupyter IP: {local_ip}")

# --- CONFIGURATION ---
MINIO_ENDPOINT = "http://minio:9000"
HIVE_METASTORE_URI = "thrift://docker-compose-hive-metastore-1:9083" # เช็คชื่อ Container Hive ดีๆ
ACCESS_KEY = "minio"
SECRET_KEY = "minio123"

PACKAGES = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0"
]

spark = (
    SparkSession.builder
    .appName("Jupyter-Iceberg-Cluster")
    
    # ชี้ไปที่ Spark Master (ใช้ชื่อ service ได้เลยเพราะอยู่ใน network เดียวกัน)
    .master("spark://spark-master:7077")
    
    # ---------------------------------------------------------
    # ✅ ส่วนสำคัญที่ต้องเพิ่มเพื่อแก้ Network Issue
    # 1. บอก Worker ว่า Driver อยู่ที่ IP ไหน
    .config("spark.driver.host", local_ip) 
    
    # 2. บังคับใช้ Port ที่เราเปิดไว้ใน Docker Compose
    .config("spark.driver.port", "20020")
    .config("spark.blockManager.port", "20021")
    
    # 3. ให้ Driver ฟังทุก Interface ใน Container ตัวเอง
    .config("spark.driver.bindAddress", "0.0.0.0")
    # ---------------------------------------------------------

    .config("spark.jars.packages", ",".join(PACKAGES))
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

    # S3 Config
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
    # Iceberg Config
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.type", "hive")
    .config("spark.sql.catalog.iceberg.uri", HIVE_METASTORE_URI)
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/iceberg_data")
    
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark Session Created with Fixed Ports!")

# ทดสอบ
spark.sql("SELECT * FROM iceberg.demo_schema.jupyter_users").show()
print("ok")

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IcebergFix") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
# ตรวจสอบว่าใช้ Python 3.11 จริงไหม
import sys
print(f"Python version: {sys.version}")

In [ ]:
spark.sql("""
    SELECT * FROM iceberg.demo_schema.jupyter_users 
""").show()

In [1]:
import os
import socket
from pyspark.sql import SparkSession

# --- 1. CONFIGURATION (ชุดเดิมที่ถูกต้อง) ---
hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)

MINIO_ENDPOINT = "http://minio:9000"
HIVE_METASTORE_URI = "thrift://docker-compose-hive-metastore-1:9083"
ACCESS_KEY = "minio"
SECRET_KEY = "minio123"

spark = (
    SparkSession.builder
    .appName("Iceberg-CRUD-Demo")
    .master("spark://spark-master:7077")
    # Network Fix
    .config("spark.driver.host", local_ip)
    .config("spark.driver.port", "20020")
    .config("spark.blockManager.port", "20021")
    .config("spark.driver.bindAddress", "0.0.0.0")

    # Packages & Extensions
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3,org.apache.hadoop:hadoop-aws:3.3.4")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

    # MinIO / S3
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
    
    # Iceberg Catalog
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.type", "hive")
    .config("spark.sql.catalog.iceberg.uri", HIVE_METASTORE_URI)
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/iceberg_data")
    
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

try:
    # ==========================================
    # 🟢 1. CREATE & INSERT (สร้างและเพิ่มข้อมูล)
    # ==========================================
    print("\n--- STEP 1: Create Table & Insert Data ---")
    
    # ลบตารางเก่าทิ้งก่อน (ถ้ามี) เพื่อเริ่มใหม่
    spark.sql("DROP TABLE IF EXISTS iceberg.demo_schema.employees")
    
    # สร้างตาราง
    spark.sql("""
        CREATE TABLE IF NOT EXISTS iceberg.demo_schema.employees (
            id INT,
            name STRING,
            role STRING,
            salary INT
        ) USING iceberg
    """)
    
    # ใส่ข้อมูลเริ่มต้น (Alice และ Bob)
    spark.sql("""
        INSERT INTO iceberg.demo_schema.employees VALUES 
        (1, 'Alice', 'Engineer', 50000),
        (2, 'Bob',   'Manager',  80000)
    """)
    
    spark.sql("SELECT * FROM iceberg.demo_schema.employees ORDER BY id").show()


    # ==========================================
    # 🟠 2. MERGE (Upsert: อัปเดตที่มีอยู่ + เพิ่มใหม่)
    # ==========================================
    print("\n--- STEP 2: MERGE (Upsert) ---")
    print("Scenario: อัปเดตเงินเดือน Bob และเพิ่มพนักงานใหม่ Charlie")

    # สร้าง Data Frame จำลองข้อมูลใหม่ที่เข้ามา
    new_data = [
        (2, 'Bob', 'Senior Manager', 90000),  # Bob (คนเดิม -> อัปเดต Role/Salary)
        (3, 'Charlie', 'Designer', 45000)     # Charlie (คนใหม่ -> เพิ่มเข้าไป)
    ]
    df_updates = spark.createDataFrame(new_data, ["id", "name", "role", "salary"])
    
    # สร้าง Temp View เพื่อใช้ใน SQL
    df_updates.createOrReplaceTempView("incoming_data")

    # คำสั่ง MERGE
    # ถ้า id ตรงกัน -> ให้อัปเดต (WHEN MATCHED)
    # ถ้า id ไม่เจอ  -> ให้เพิ่มใหม่ (WHEN NOT MATCHED)
    spark.sql("""
        MERGE INTO iceberg.demo_schema.employees AS target
        USING incoming_data AS source
        ON target.id = source.id
        WHEN MATCHED THEN 
            UPDATE SET target.role = source.role, target.salary = source.salary
        WHEN NOT MATCHED THEN 
            INSERT (id, name, role, salary) VALUES (source.id, source.name, source.role, source.salary)
    """)

    spark.sql("SELECT * FROM iceberg.demo_schema.employees ORDER BY id").show()


    # ==========================================
    # 🔴 3. DELETE (ลบข้อมูลตามเงื่อนไข)
    # ==========================================
    print("\n--- STEP 3: DELETE ---")
    print("Scenario: ลบพนักงานที่ชื่อ Alice ออก")

    spark.sql("""
        DELETE FROM iceberg.demo_schema.employees 
        WHERE name = 'Alice'
    """)

    # ดูผลลัพธ์สุดท้าย (ควรเหลือ Bob และ Charlie)
    final_df = spark.sql("SELECT * FROM iceberg.demo_schema.employees ORDER BY id")
    final_df.show()

    print(f"Final Row Count: {final_df.count()}")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    # spark.stop()
    print("\n✅ Session Closed.")


--- STEP 1: Create Table & Insert Data ---
+---+-----+--------+------+
| id| name|    role|salary|
+---+-----+--------+------+
|  1|Alice|Engineer| 50000|
|  2|  Bob| Manager| 80000|
+---+-----+--------+------+


--- STEP 2: MERGE (Upsert) ---
Scenario: อัปเดตเงินเดือน Bob และเพิ่มพนักงานใหม่ Charlie
+---+-------+--------------+------+
| id|   name|          role|salary|
+---+-------+--------------+------+
|  1|  Alice|      Engineer| 50000|
|  2|    Bob|Senior Manager| 90000|
|  3|Charlie|      Designer| 45000|
+---+-------+--------------+------+


--- STEP 3: DELETE ---
Scenario: ลบพนักงานที่ชื่อ Alice ออก
+---+-------+--------------+------+
| id|   name|          role|salary|
+---+-------+--------------+------+
|  2|    Bob|Senior Manager| 90000|
|  3|Charlie|      Designer| 45000|
+---+-------+--------------+------+

Final Row Count: 2

✅ Session Closed.


In [2]:
spark.sql("""
    SELECT * FROM iceberg.demo_schema.jupyter_users 
""").show()

+---+-----------+--------------------+
| id|   username|          created_at|
+---+-----------+--------------------+
|  4|test_insert|2026-01-31 09:06:...|
|  1|       test|2026-01-31 08:51:...|
|  2|Spark_Ninja|2026-01-31 08:51:...|
|  3|Hive_Master|2026-01-31 08:51:...|
+---+-----------+--------------------+

